# analysing-algebra-coding-challenge
A coding challenge around implementing a command-line RPN calculator, and then converting algebraic expressions into RPN format for calculation.

## Core Challenge
Reverse Polish Notation (RPN) is a way of encoding algebra that was made most famous by HP (Hewlett Packard as was) calculators.  For the purposes of this challenge, an RPN expression is a set of lines of text containing whitespace-separated tokens.  The tokens can be either a decimal number or one of the four arithmetic operators '+' (plus), '-' (minus), '*' (times or multiplied by) or '/' (divided by).

When the token is a decimal number, it is pushed onto a stack.  When the token is an arithmetic, the two most recent entries on the stack are removed from the stack, the operator is applied to them, and then the result is pushed onto the stack.

When a new line is encountered, or when the end of the last line is encountered, the contents of the stack should be printed out, from oldest entry to newest.

Implement a command-line RPN calculator as described above.

### Examples
* ```"3 5"``` is printed out as: ```3 5```
* ```"2.2 3.3 +"``` is printed out as: ```5.5```
* ```"2 -5 -"``` is printed out as: ```7```
* ```"8 6 + 4 *"``` is printed out as: ```56```
* ```"3 5 + 3 1 + *"``` is printed out as: ```32```
* ```"9 9 + 3 /"``` is printed out as: ```6```
* ```"5 2 /"``` is printed out as: ```2.5```

It is an error if there are fewer than two numbers in the stack when an arithmetic operator is encountered, and it is an error to divide by zero.

### Solution to Core Challenge

The RPN algebraic format is simple enough that we can write the parsing code directly, without needing to use a parser libary.

Here it is convenient to use F#'s 'Active Patterns', that provide for rich pattern matching.

In [1]:
let (|Number|_|) (str: string) =
   let mutable floatvalue = 0.0
   if System.Double.TryParse(str, &floatvalue) then Some(floatvalue)
   else None

In [4]:
let (|Operator|_|) (str: string): char option =
    match str with
    | "+" | "-" | "*" | "/" -> Some(str.[0])
    | unknown -> None

In [7]:
type Stack = double list

let toStackString (stack:Stack) =
    sprintf "%s" (stack |> List.rev |> List.map (fun (dbl) -> dbl.ToString()) |> List.toSeq |> String.concat " ")

let printStack (stack:Stack) =
    printfn "%s" (toStackString stack)

In [37]:
open System.Text.RegularExpressions

// Process single token
let rpntoken (stack: Stack) (token: string): Stack =
    match token with
    | Number dbl -> dbl::stack
    | Operator op ->
        match stack with
        | op2::op1::tail ->
            match op with
            | '+' -> (op1+op2)::tail
            | '-' -> (op1-op2)::tail
            | '*' -> (op1*op2)::tail
            | '/' ->
                match op2 with
                | 0. -> failwith (sprintf "Tried to divide by zero => %s" (toStackString stack))
                | nonzero -> (op1/op2)::tail
        | toofewelements -> failwith (sprintf "Too few elements on stack for %c => %s" op (toStackString stack))
    | unknown -> failwith (sprintf "Unrecognised token: %s => %s" unknown (toStackString stack))

// Single-line RPN calculation
let rpncalc1 (stack: Stack) (singlelinestr: string): Stack =
    let result = (
        Regex.Matches(singlelinestr, @"\S+") |>
        Seq.map(fun (mtch) -> mtch.ToString()) |>
        Seq.fold rpntoken stack
    )
    do printStack result
    result

In [56]:
let rpncalc (multilinestr: string) =
    ignore (multilinestr.Split("\n") |> Seq.filter(fun (s) -> s.Length > 0) |> Seq.fold rpncalc1 [])

### Test the Solution

In [57]:
ignore (rpncalc "3 5") // 3 5

3 5


In [58]:
ignore (rpncalc "2.2 3.3 +") // 5.5

5.5


In [59]:
ignore (rpncalc "2 -5 -") // 7

7


In [60]:
ignore (rpncalc "8 6 + 4 *") // 56

56


In [61]:
ignore (rpncalc "3 5 + 3 1 + *") // 32

32


In [62]:
ignore (rpncalc "9 9 + 3 /") // 6

6


In [63]:
ignore (rpncalc "5 2 /") // 2.5

2.5


In [64]:
ignore (rpncalc """
3 5
+
4 *
5 3 +
/
""")


3 5
8
32
32 8
4



## Stretch Challenge
The more common way to write arithmetic expressions is with numbers, operators and also parentheses, e.g.
* (2+3) * (4*5)
* (10/5)*5
* 10/(5*5)

Write a command-line converter that converts arbitrary arithmetic expressions into RPN expressions.